In [1]:
import numpy as np
import gp_grief
from scipy.stats import multivariate_normal as mvn #.logpdf
from scipy.stats import chi2
import pymcmc as pm
from numpy.testing import *
from pdb import set_trace
gp_grief.debug()

Initialize the dataset

In [2]:
np.random.seed(0)
X = np.random.randn(100, 4)
X[:, 0] = 1. # Intercept column.
Y = np.dot(X, [0.5, 0.1, 0.25, 1.]) + 0.1 * np.random.randn(X.shape[0])

Check the log-likelihood value

In [3]:
np.random.seed(0)
m = gp_grief.models.GPweb_transformed(Phi=X, y=Y)
tmp = np.random.rand(*m.parameters.shape) + 1e-6
m.parameters = tmp
ll_gpweb = m.log_likelihood()

# now compute manually
w = m.kern.parameters.reshape((1,-1))
sig2 = m.noise_var
Phit = np.linalg.svd(X, full_matrices=False, compute_uv=True)[0]
K = Phit.dot(np.diag(w.squeeze()).dot(Phit.T)) + sig2*np.identity(Phit.shape[0])
ll_exact = mvn.logpdf(Y.squeeze(), mean=np.zeros(Phit.shape[0]), cov=K)

assert_array_almost_equal(ll_gpweb, ll_exact)

[ 14:27:04 ] gp_grief.models DEBUG: Initializing GPweb_transformed model.


check the log likelihood-gradients

In [4]:
m.checkgrad()

[ 14:27:04 ] gp_grief.models INFO: Gradient check passed.


True

now wrap in pymcmc

In [5]:
mcm = gp_grief.models.PyMCMC_Wrapper(m)
print mcm
print "Printing the transformed parameters:"
print mcm.params 

# perform the check
mcm.checkgrad()


GPweb_transformed Model
| Name      |    Value | Constraint   |
|-----------+----------+--------------|
| noise_var | 0.548815 | +ve          |
Printing the transformed parameters:
[-0.31306898  0.04361093 -0.18975376 -0.32239961 -0.63953869]
[ 14:27:04 ] gp_grief.models INFO: Gradient check passed.


and now sample

In [6]:

# # Pick a proposal for MCMC (here we pick a Metropolized Langevin Proposal
# proposal = pm.MALAProposal()
# # Construct a Metropolis Hastings object
# mcmc = pm.MetropolisHastings(mcm, # The model you want to train
#               proposal=proposal,         # The proposal you want to use
#               db_filename=None)#'demo_1_db.h5')# The HDF5 database to write the results
# # Now we can sample it:
# iters = 10000
# n_thin = int(max(50,   iters//1000)) # save at most 1000 samples
# n_burn = int(max(1000, iters//10)) # burn no less than 10% of the total samples
# print("running MCMC. n_sample=%d, n_thin=%d, n_burn=%d"%(iters,n_thin,n_burn))
# chain = mcmc.sample(int(iters),         # Number of MCMC steps
#         num_thin=n_thin,  # Number of steps to skip
#         num_burn=n_burn,  # Number of steps to burn initially
#         verbose=True)   # Be verbose or not
# print "final sampled basis fun weight stats: min=%.4g, max=%.4g, ptp=%.4g, mean=%.4g, std=%.4g" % \
#     tuple([fcn(chain[:,1:]) for fcn in [np.min, np.max, np.ptp, np.mean, np.std]])